<a href="https://colab.research.google.com/github/humanet1603/week1/blob/main/%EB%B3%B4%EC%88%98%EC%9C%A0%ED%8A%9C%EB%B8%8C%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

# API 키를 수동으로 입력 (입력 내용은 화면에 보이지 않도록 처리 가능)
api_key = input("🔑 YouTube API 키를 입력하세요: ").strip()

# 환경변수로 설정
os.environ["YOUTUBE_API_KEY"] = api_key

# 이후 사용 시
print("✅ API 키가 성공적으로 설정되었습니다.")


In [ ]:
from googleapiclient.discovery import build
import os
from datetime import datetime, timedelta

# 1. API 키 가져오기
api_key = os.getenv("YOUTUBE_API_KEY")
youtube = build("youtube", "v3", developerKey=api_key)

# 2. 성창경TV 채널 ID
channel_id = "UCFWIofVBqseX6Bmd9Dn7kgg"

# 3. 날짜 필터 설정
# 예: 2025년 3월 24일 ~ 3월 30일
start_date = "2025-03-24T00:00:00Z"
end_date   = "2025-03-30T23:59:59Z"

# 4. 영상 검색 및 메타데이터 수집 함수
def get_videos_with_metadata(channel_id, start_date, end_date, max_results=50):
    request = youtube.search().list(
        part="snippet",
        channelId=channel_id,
        publishedAfter=start_date,
        publishedBefore=end_date,
        maxResults=max_results,
        order="date",
        type="video"
    )
    response = request.execute()

    video_data = []
    for item in response['items']:
        video_id = item['id']['videoId']
        title = item['snippet']['title']
        published_at = item['snippet']['publishedAt']
        description = item['snippet']['description']
        video_url = f"https://www.youtube.com/watch?v={video_id}"

        # 각 영상의 통계(조회수) 가져오기
        stats_request = youtube.videos().list(
            part="statistics",
            id=video_id
        )
        stats_response = stats_request.execute()
        view_count = stats_response['items'][0]['statistics'].get('viewCount', '0')

        video_data.append({
            "title": title,
            "published_at": published_at,
            "view_count": view_count,
            "description": description,
            "url": video_url
        })

    return video_data

# 5. 실행
videos = get_videos_with_metadata(channel_id, start_date, end_date)

# 6. 출력
print(f"\n📺 성창경TV 영상 목록 ({start_date[:10]} ~ {end_date[:10]})\n")
for i, video in enumerate(videos, 1):
    print(f"{i}. {video['title']}")
    print(f"   🗓 게시일: {video['published_at']}")
    print(f"   👁 조회수: {video['view_count']}")
    print(f"   📄 설명: {video['description'][:100]}{'...' if len(video['description']) > 100 else ''}")
    print(f"   🔗 링크: {video['url']}\n")
